In [30]:
import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

import os
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, Callback


# Set memory growth for GPU devices
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.13.0
Hub version: 0.14.0
GPU is available


In [31]:
train_directory='../input/100-bird-species/train'
val_directory='../input/100-bird-species/valid'
test_directory='../input/100-bird-species/test'


train_datagen=ImageDataGenerator(
 rescale=1/255.0, 
 rotation_range=10, 
 zoom_range=0.05, 
 width_shift_range=0.05, 
 height_shift_range=0.05, 
 shear_range=0.05,
 horizontal_flip=True,
 fill_mode='nearest')

batch_size =64

train_generator = train_datagen.flow_from_directory(
    directory=train_directory,
    target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True, 
    seed=42
)
val_datagen=ImageDataGenerator(rescale=1/255.0)
valid_generator = val_datagen.flow_from_directory(
    directory=val_directory,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_datagen = ImageDataGenerator(rescale=1/255.0) 
test_generator = test_datagen.flow_from_directory(
  directory=test_directory,
  target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [32]:

# Training
steps_per_epoch=train_generator.n//train_generator.batch_size
validation_steps=valid_generator.n//valid_generator.batch_size
test_steps=test_generator.n//test_generator.batch_size
epochs = 50
lr = 0.0001

# Create the model
model = tf.keras.Sequential([
    # Pre-trained ResNet-50 model
    tf.keras.Input(shape=(224,224,3)),
    hub.KerasLayer("https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1", trainable=False),
    tf.keras.layers.Flatten(),

    # Additional layers
    # tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1024, activation='relu'),
    # tf.keras.layers.Dropout(rate=0.2),
    # tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    # Output Dense layer
    tf.keras.layers.Dense(525,  activation= 'softmax')
])



# model.build([None, 224, 224, 3])
# model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr),metrics=['acc'])

model.summary()

Attaching model 'tensorflow/resnet-50/tensorflow2/feature-vector/1' to your Kaggle notebook...


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_7 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 flatten_7 (Flatten)         (None, 2048)              0         
                                                                 
 dense_14 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_7 (Dropout)         (None, 1024)              0         
                                                                 
 dense_15 (Dense)            (None, 525)               538125    
                                                                 
Total params: 26197453 (99.94 MB)
Trainable params: 2636301 (10.06 MB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________


In [33]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# Instantiate a metric to track accuracy
accuracy_metric = tf.keras.metrics.CategoricalAccuracy()

# Instantiate early stopping and model checkpoint parameters
early_stopping = {'patience': 3, 'best_loss': float("inf")}
model_checkpoint = {'filepath': '/kaggle/working/best_model.h5'}


for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    # Reset the accuracy at the start of each epoch
    accuracy_metric.reset_states()
    
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_generator):
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * batch_size))
        if step >= steps_per_epoch:
            break
    # Calculate and print the accuracy at the end of each epoch
    train_accuracy = accuracy_metric.result().numpy()
            
    print("Training accuracy at the end of epoch %d: %.4f" % (epoch, train_accuracy))
    
    # Assuming you have validation data
    # Evaluate on the validation set without using model.evaluate
    accuracy_metric.reset_states()
    validation_loss = 0.0  # Initialize validation loss

    for step, (x_batch_train, y_batch_train) in enumerate(valid_generator):
        val_logits = model(x_batch_val, training=False)
        val_loss = loss_fn(y_batch_val, val_logits)

        # Accumulate the validation loss
        validation_loss += val_loss.numpy()

        accuracy_metric.update_state(y_batch_val, val_logits)
        if step >= validation_steps:
            break

    # Compute the mean validation loss
    validation_loss /= len(valid_generator)

    validation_accuracy = accuracy_metric.result().numpy()

    print("Validation loss: %.4f" % validation_loss)
    print("Validation accuracy: %.4f" % validation_accuracy)
    

    # Assuming you have validation data
    # Evaluate on the validation set without using model.evaluate
    accuracy_metric.reset_states()
    test_loss = 0.0  # Initialize validation loss

    for step, (x_batch_train, y_batch_train) in enumerate(test_generator):
        test_logits = model(x_batch_val, training=False)
        test_loss = loss_fn(y_batch_val, val_logits)

        # Accumulate the validation loss
        test_loss += test_loss.numpy()

        accuracy_metric.update_state(y_batch_val, val_logits)
        if step >= test_steps:
            break

    # Compute the mean validation loss
    test_loss /= len(test_generator)

    test_accuracy = accuracy_metric.result().numpy()
    
    print("Test loss: %.4f" % test_loss)
    print("Test accuracy: %.4f" % test_accuracy)
    

    # Check if the validation loss has improved
    if validation_loss < early_stopping['best_loss']:
        early_stopping['best_loss'] = validation_loss
        early_stopping['wait'] = 0  # Reset the patience counter
        model.save_weights(model_checkpoint['filepath'])
        print("Saved model weights to '%s'" % model_checkpoint['filepath'])
    else:
        early_stopping['wait'] += 1
        print("No improvement in validation loss. Patience: %d/%d" % (early_stopping['wait'], early_stopping['patience']))

    # Check if early stopping criteria are met
    if early_stopping['wait'] >= early_stopping['patience']:
        print("Early stopping criterion reached. Stopping training.")
        break

        



Start of epoch 0
Training loss (for one batch) at step 0: 6.5712
Seen so far: 64 samples
Training loss (for one batch) at step 200: 4.6014
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 2.9377
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 2.2987
Seen so far: 38464 samples
Training loss (for one batch) at step 800: 1.7580
Seen so far: 51264 samples
Training loss (for one batch) at step 1000: 1.5953
Seen so far: 64064 samples
Training loss (for one batch) at step 1200: 1.1409
Seen so far: 76864 samples
Training accuracy at the end of epoch 0: 0.4584
Validation loss: 0.9204
Validation accuracy: 0.8125
Test loss: 0.0438
Test accuracy: 0.8125
Saved model weights to '/kaggle/working/best_model.h5'

Start of epoch 1
Training loss (for one batch) at step 0: 1.2060
Seen so far: 64 samples
Training loss (for one batch) at step 200: 1.0941
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 0.9448
Seen so far: 25664 samples
Trainin

In [34]:
# # Define callbacks


# class CustomCallback(Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         if (epoch + 1) % 5 == 0:
#             val_acc = self.model.evaluate(test_generator, return_dict=True)['acc']
#             print(f'test accuracy at epoch {epoch + 1}: {val_acc}')

#             self.model.save(f'/kaggle/working/models/model_val_acc_{epoch + 1}.h5')

# custom_callback = CustomCallback()




# model.fit(
#     train_generator,
#     epochs=epochs,
#     steps_per_epoch=steps_per_epoch,
#     validation_data=valid_generator,
#     validation_steps=validation_steps,
#     callbacks=[tensorboard_callback, custom_callback]
# )



In [35]:
# results = model.evaluate(test_generator, verbose=0)

# print("    Test Loss: {:.5f}".format(results[0]))
# print("Test Accuracy: {:.2f}%".format(results[1] * 100))